In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as img
# import torchvision.transforms as transforms

In [11]:
df = pd.read_csv('train.csv')
df = df.loc[df['number'] == 5].drop(columns = 'number') 
print(df.shape)

(3795, 784)


In [12]:
x_mean = torch.tensor(np.mean(np.array(df),axis=0,keepdims=True).reshape([1,784]),dtype=torch.float)
x_std = torch.tensor((np.std((np.array(df) + 1e-4),axis=0,keepdims=True)**-1).reshape([1,784]),dtype=torch.float)

In [13]:
class generator (nn.Module):
    def __init__(self):
        super(generator,self).__init__()
        self.fwd = nn.Sequential(
            nn.Linear(64,256),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(256,28*28),
            nn.Tanh())
    def forward (self,noise):
        img = self.fwd(noise)
        return img

In [14]:
class critic (nn.Module):
    def __init__(self):
        super(critic,self).__init__()
        self.fwd = nn.Sequential(
            nn.Linear(in_features=28*28,out_features=128),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(in_features=128,out_features=1),
            nn.ReLU())
    def forward (self,img):
        out = self.fwd(img)
        return out

In [15]:
g = generator()
c = critic()

In [16]:
g_optimizer = torch.optim.RMSprop(g.parameters(), lr=1e-3)
c_optimizer = torch.optim.RMSprop(c.parameters(), lr=1e-4)

In [17]:
PATH1 = "generator.pt"
g.load_state_dict(torch.load(PATH1))
PATH2 = "critic.pt"
c.load_state_dict(torch.load(PATH2))

<All keys matched successfully>

In [18]:
for j in range (70):
   for i in range (237):
      img = (torch.tensor(np.array(df.iloc[16*i:16*(i+1)]).reshape([16,784]),dtype=torch.float) - x_mean)*x_std
      noise = torch.tensor(np.random.rand(16,64),dtype=torch.float)
      fake_img1 = g(noise)
      noise = torch.tensor(np.random.rand(32,64),dtype=torch.float)
      fake_img2 = g(noise)

      x_cap = img*(0.3) + fake_img1*(0.7)
      interp_logits = c(x_cap)
      grad_outputs = torch.ones_like(interp_logits)
      gradients = torch.autograd.grad(outputs=interp_logits, inputs=x_cap, grad_outputs=grad_outputs, create_graph=True, retain_graph=True)[0]
      gradients = gradients.view(16, -1)
      grad_norm = gradients.norm(2, 1)

      loss = -(torch.mean(c(fake_img1) )- torch.mean(c(img)) + (torch.mean((grad_norm - 1) ** 2))*10)
      c_optimizer.zero_grad()
      loss.backward()
      c_optimizer.step()

      out = c(fake_img2)
      loss = -(torch.mean(c(fake_img2)).view(-1))
      g_optimizer.zero_grad()
      loss.backward()
      g_optimizer.step()

   print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [19]:
PATH1 = "generator.pt"
torch.save(g.state_dict(), PATH1)
PATH2 = "critic.pt"
torch.save(c.state_dict(), PATH2)

In [20]:
n = torch.tensor(np.random.rand(1,64),dtype=torch.float)
# print(g(n))
pic = ((g(n) * (1/x_std)) + x_mean).detach().numpy().reshape([28,28])
# print(pic)
cv.imwrite('number.png',pic)

True

In [13]:
testImage = cv.imread('number.png')
plt.imshow(testImage)

: 

: 